In [ ]:
!pip install wandb

In [58]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
import wandb
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

### Wandb

In [ ]:
wandb.init(project="actor_Deepface")

In [60]:
cfg = wandb.config
cfg.learning_rate = 0.001

In [62]:
!pip install deepface
from deepface import DeepFace

### Import data

In [ ]:
emb = []
c = []
for i in range(1,3):
  for file in os.listdir(f'./data/{i}'):
    embedding = DeepFace.represent(img_path = f"data/{i}/{file}", model_name = 'ArcFace')    
    emb.append(embedding)
    c.append(i)

data = pd.DataFrame(emb)
data["class"] = c
data.head

In [64]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,class
0,-0.268964,0.654269,-0.180147,-0.437032,-0.034369,0.408288,-0.214968,-0.223740,-0.004572,-0.055405,...,0.208208,0.465215,-0.900044,0.352018,-0.344363,-0.011765,-0.035854,-0.560585,-1.227006,1
1,0.074446,0.284225,-0.073452,-0.055838,-0.201645,0.180380,0.058122,-0.075720,-0.247919,-0.086424,...,0.314023,0.022977,0.036643,-0.060317,0.148403,0.004093,0.241785,-0.097623,0.175796,1
2,-0.291828,0.841481,-0.217782,-0.678744,-0.029751,0.395901,-0.223595,-0.099625,0.002754,-0.162070,...,0.295298,0.520281,-0.899359,0.368394,-0.387302,-0.044264,0.151620,-0.425838,-1.343823,1
3,-0.093858,0.037026,-0.009606,-0.182945,-0.138191,0.055743,0.009191,-0.025234,0.084224,0.106737,...,0.048316,0.046861,-0.082306,0.027742,-0.115165,-0.039259,0.104289,0.157615,0.019476,1
4,-0.084899,0.420953,-0.228951,0.005544,-0.244540,0.400676,0.175215,-0.061855,-0.096766,-0.165337,...,0.443058,0.042381,-0.020578,0.137263,0.277856,-0.263791,0.073567,0.031678,-0.065310,1


In [65]:
data.to_csv('deepfaceData.csv', index=False)

In [66]:
X = data.iloc[0:,0:512]
Y = data["class"]
Y.shape

(91,)

In [67]:
X_train, X_test , Y_train , Y_test = train_test_split(X ,Y ,shuffle=True)

### Model

In [68]:
model = tf.keras.models.Sequential([
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dense(3, activation="softmax")
])

In [69]:
model.compile(optimizer=tf.keras.optimizers.Adam(cfg.learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
              )

### Fit

In [70]:
model.fit(X_train, Y_train, epochs=180, callbacks=[WandbCallback()])

wandb: WARNING Unable to compute FLOPs for this model.


Epoch 1/180
3/3 [==============================] - 1s 11ms/step - loss: 1.0279 - accuracy: 0.3382 - _timestamp: 1659289587.0000 - _runtime: 129.0000
Epoch 2/180
3/3 [==============================] - 0s 8ms/step - loss: 0.7311 - accuracy: 0.6618 - _timestamp: 1659289587.0000 - _runtime: 129.0000
Epoch 3/180
3/3 [==============================] - 0s 7ms/step - loss: 0.5725 - accuracy: 0.7794 - _timestamp: 1659289587.0000 - _runtime: 129.0000
Epoch 4/180
3/3 [==============================] - 0s 9ms/step - loss: 0.4779 - accuracy: 0.6912 - _timestamp: 1659289587.0000 - _runtime: 129.0000
Epoch 5/180
3/3 [==============================] - 0s 8ms/step - loss: 0.4343 - accuracy: 0.6765 - _timestamp: 1659289587.0000 - _runtime: 129.0000
Epoch 6/180
3/3 [==============================] - 0s 8ms/step - loss: 0.3651 - accuracy: 0.8088 - _timestamp: 1659289587.0000 - _runtime: 129.0000
Epoch 7/180
3/3 [==============================] - 0s 10ms/step - loss: 0.3687 - accuracy: 0.7941 - _timestamp:

### Evaluate

In [72]:
model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 22ms/step - loss: 0.7028 - accuracy: 0.9565


[0.702785849571228, 0.95652174949646]